In [1]:
import pandas as pd
from load_datasets import path_to_datasets
combined_dataset = None

with open('/' + path_to_datasets + '/combined_dataset.pkl', 'rb') as f:
    combined_dataset = pickle.load(f)

ModuleNotFoundError: No module named 'en_core_web_sm'

In [ ]:
def_test_lf(lf, data):
    